# Alimentador obra - ubicación

Notebook para completar la ubicación de obras con ciudad y país anterior

In [50]:
import json
import numpy as np
import requests
import pandas as pd

## Abrir tablas y filtrar datos



In [51]:
config = json.load(open('arca-nov29-config.json'))
rename_dict = config['fields_map']
obrascsv = '../tablas/tablas-arcanov29/Registro general.csv'
ubsl = pd.read_csv('../tablas/tablas-arcanov29/Ubicacion_1_ Lista.csv')
ubsl = ubsl.dropna(how='all')
ubsl.dropna(subset=['id'])
# dataframes de las tablas
obras = pd.read_csv(obrascsv, dtype=str)
obras = obras.rename(columns=rename_dict)
obras = obras.dropna(how='all')
obras = obras.fillna('')
print(len(obras.index))

ubicaciones = obras.drop_duplicates(subset='id_ubicacion')
print(f'ubicaciones en obra: {len(ubicaciones.index)}')
print(f'Ubicaciones en lista: {len(ubsl.index)}')
ciudades = obras.drop_duplicates(subset='Ciudades_actual_id')
print(f'Ciudades en obras:{len(ciudades.index)}')
paises = obras.drop_duplicates(subset='Paises_actual_id')
print(f'Paises en obras: {len(paises.index)}')

24999
ubicaciones en obra: 1929
Ubicaciones en lista: 1938
Ciudades en obras:651
Paises en obras: 41


In [34]:
#ubicaciones = obras[[ 'Ciudades_origen_id', 'Ciudad Origen', 'Ciudades_actual_id',
 #      'Ciudad Actual', 'Paises_origen_id', 'Pais Origen', 'Paises_actual_id',
 #      'Pais Actual', 'id_ubicacion', 'Ubicacion']]

#ubs = ubicaciones.groupby(['Pais Actual','Ciudad Actual','Ubicacion'])
#len(ubs.groups)
#ubs = ubicaciones.groupby('Paises_actual_id').apply(lambda a: dict(a.groupby('Ciudades_actual_id').apply(lambda x: dict(zip(x['id_ubicacion'])))))
#ubs.describe()


Ciudades_origen_id  \
                                                                                               count   
Pais Actual Ciudad Actual      Ubicacion                                                               
                                                                                                   1   
Alemania    Berlin             Ethnologisches Museum (Museo Etnológico, Berlín)                    1   
            Hannover           Landesmuseum (Museo Nacional de Hannover)                           1   
Argentina   Buenos Aires       Arzobispado de Buenos Aires (Buenos Aires)                          1   
                               Basilica Nuestra señora del Rosario - Convento ...                  1   
...                                                                                              ...   
Venezuela   Tocuyo             Iglesia del Rio Tocuyo (Rio Tocuyo)                                 1   
                               Museo del Tocuyo y Lisandro Alvarado (Tocuyo)                       1   
            Táchira            Cátedra Santo Tomás de Aquino                                       1   
            Valencia (Vn)      Iglesia san Blas (Valencia, Carabobo)                               1   
            Valle de la Pascua Iglesia Nuestra Señora de la Encarnación (El Va...                  1   

                                                                                          \
                                                                                  unique   
Pais Actual Ciudad Actual      Ubicacion                                                   
                                                                                       1   
Alemania    Berlin             Ethnologisches Museum (Museo Etnológico, Berlín)        1   
            Hannover           Landesmuseum (Museo Nacional de Hannover)               1   
Argentina   Buenos Aires       Arzobispado de Buenos Aires (Buenos Aires)              1   
                               Basilica Nuestra señora del Rosario - Convento ...      1   
...                                                                                  ...   
Venezuela   Tocuyo             Iglesia del Rio Tocuyo (Rio Tocuyo)                     1   
                               Museo del Tocuyo y Lisandro Alvarado (Tocuyo)           1   
            Táchira            Cátedra Santo Tomás de Aquino                           1   
            Valencia (Vn)      Iglesia san Blas (Valencia, Carabobo)                   1   
            Valle de la Pascua Iglesia Nuestra Señora de la Encarnación (El Va...      1   

                                                                                        \
                                                                                   top   
Pais Actual Ciudad Actual      Ubicacion                                                 
                                                                                         
Alemania    Berlin             Ethnologisches Museum (Museo Etnológico, Berlín)    144   
            Hannover           Landesmuseum (Museo Nacional de Hannover)           118   
Argentina   Buenos Aires       Arzobispado de Buenos Aires (Buenos Aires)          159   
                               Basilica Nuestra señora del Rosario - Convento ...  159   
...                                                                                ...   
Venezuela   Tocuyo             Iglesia del Rio Tocuyo (Rio Tocuyo)                 143   
                               Museo del Tocuyo y Lisandro Alvarado (Tocuyo)       143   
            Táchira            Cátedra Santo Tomás de Aquino                       676   
            Valencia (Vn)      Iglesia san Blas (Valencia, Carabobo)               672   
            Valle de la Pascua Iglesia Nuestra Señora de la Encarnación (El Va...  588   

                                                                           

## URLS

In [54]:
#baseurl = 'http://localhost:8055/'
baseurl = config['baseurl']
#obra_simbolos_baseurl = f'{baseurl}items/obra_simbolos_lista'
obras_url = f'{baseurl}items/obra'
#simbolos_url = f'{baseurl}items/simbolos_lista'
paises_url = f'{baseurl}items/paises_lista/'
ciudades_url = f'{baseurl}items/ciudades_lista/'
ubicaciones_url = f'{baseurl}items/ubicacion/'
key = config['key']
headers =  {'Authorization':f'Bearer {key}'}


## Iteración Ciudades País

Creación de relación entre ciudad y país. Hace una peticion a la API buscando una ciudad con el ```arca_id```.  Hace otra petición buscando el país usando su ```arca_id```. Si encuentra las dos, hace una peticion a la lista de ciudades, estableciendo su relación con el país.


In [ ]:
for index, row in ciudades.iterrows():
    obra_arcaID = row['arca_id']

    ciudad_actual = row['Ciudad Actual']
    ca_id = row['Ciudades_actual_id']
    #ciudad_anterior = ['Ciudades']
    pais_actual = row['Pais Actual']
    #pais actual id
    pa_id = row['Paises_actual_id']
    #pais endpoint
    pa_ep = f'{paises_url}?filter[arca_id][_eq]={pa_id}&limit=1'
    ca_ep = f'{ciudades_url}?filter[arca_id][_eq]={ca_id}&limit=1'

    #obtener pais de la api
    try:
        r_pais = requests.get(pa_ep)
        r_pais.raise_for_status()
        print(f'econtrada pais: {pa_id}')
    except requests.RequestException as err:
        print(f'Error obteniendo pais: {pa_id}')
        print(err)
        # si se produce error, continua el siguiente registro
        continue

    #se obtienen los datos de la respuesta
    try:
        pais = r_pais.json()['data'][0]
    except IndexError as err:
        print(f'Pais no encontrado:{pa_id}')
        continue

    print(pais)
    #obtener ciudad de la api
    try:
        r_ciudad = requests.get(ca_ep)
        r_ciudad.raise_for_status()
        print(f'econtrada ciudad: {ca_id}')
    except requests.RequestException as err:
        print(f'Error obteniendo ciudad: {ca_id}')
        print(err)
        # si se produce error, continua el siguiente registro
        continue

    #se obtienen los datos de la respuesta
    try:
        ciudad = r_ciudad.json()['data'][0]
    except IndexError as err:
        print(f'Ciudad no encontrada: {ca_id}')
        continue
    
    print(ciudad)

    # crear el objeto obra simbolo con los ids de los objetos viculados
    ciudad['pais'] = pais['id']
    
    # actualizar pais
    ciudad_id = ciudad['id']
    ciudad_ep = f'{ciudades_url}{ciudad_id}'
    try:
        r_ciudadup = requests.patch(ciudad_ep, json = ciudad, headers=headers)
        r_ciudadup.raise_for_status()
    except requests.RequestException as err:
        print(f'Error actualizando ciudad: id:{ciudad_id}, arca_id:{ca_id}')
        print(err)
        continue

    ciudadup = r_ciudadup.json()['data']
    print('Creada relacion ciudad pais')
    print(ciudadup)
    

## Iteración Ubicaciones

Creación de relación entre ubicación y país. Hace una peticion a la API buscando una ubicación con su ```arca_id```.  Hace otra petición buscando la ciudad usando su ```arca_id```. Si encuentra las dos, hace una peticion a la lista de ciudades, estableciendo su relación con el la ciudad.


In [57]:
for index, row in ubicaciones.iterrows():
    obra_arcaID = row['arca_id']

    ciudad_actual = row['Ciudad Actual']
    ca_id = row['Ciudades_actual_id']
    #ciudad_anterior = ['Ciudades']
    ubi_actual = row['Pais Actual']
    #pais actual id
    ua_id = row['id_ubicacion']
    #pais endpoint
    ua_ep = f'{ubicaciones_url}?filter[arca_id][_eq]={ua_id}&limit=1'
    ca_ep = f'{ciudades_url}?filter[arca_id][_eq]={ca_id}&limit=1'

    #obtener pais de la api
    try:
        r_ubi = requests.get(ua_ep)
        r_ubi.raise_for_status()
        print(f'econtrada ubicacion: {ua_id}')
    except requests.RequestException as err:
        print(f'Error obteniendo pais: {ua_id}')
        print(err)
        # si se produce error, continua el siguiente registro
        continue

    #se obtienen los datos de la respuesta
    try:
        ubicacion = r_ubi.json()['data'][0]
    except IndexError as err:
        print(f'ubicacion no encontrada:{ua_id}')
        continue

    print(pais)
    #obtener ciudad de la api
    try:
        r_ciudad = requests.get(ca_ep)
        r_ciudad.raise_for_status()
        print(f'econtrada ciudad: {ca_id}')
    except requests.RequestException as err:
        print(f'Error obteniendo ciudad: {ca_id}')
        print(err)
        # si se produce error, continua el siguiente registro
        continue

    #se obtienen los datos de la respuesta
    try:
        ciudad = r_ciudad.json()['data'][0]
    except IndexError as err:
        print(f'Ciudad no encontrada: {ca_id}')
        continue
    
    print(ciudad)

    # crear el objeto obra simbolo con los ids de los objetos viculados
    ubicacion['ciudad'] = ciudad['id']
    
    # actualizar pais
    ubicacion_id = ubicacion['id']
    ubicacion_ep = f'{ubicaciones_url}{ubicacion_id}'
    try:
        r_ubicacionup = requests.patch(ubicacion_ep, json = ubicacion, headers=headers)
        r_ubicacionup.raise_for_status()
    except requests.RequestException as err:
        print(f'Error actualizando ubicacion: id:{ubicacion_id}, arca_id:{ca_id}')
        print(err)
        continue

    ubicacionup = r_ubicacionup.json()['data']
    print('Creada relacion ubicacion pais')
    print(ciudadup)
    

econtrada ubicacion: 344
{'id': 17, 'status': 'draft', 'sort': None, 'user_created': 'eb3d7a52-0464-40f5-aff3-37009b78ad61', 'date_created': '2021-12-07T21:52:26.419Z', 'user_updated': None, 'date_updated': None, 'arca_id': 17, 'codigo_iso': 'MX', 'nombre': None, 'nombre_es': 'México'}
econtrada ciudad: 131
{'id': 131, 'status': 'draft', 'sort': None, 'user_created': 'eb3d7a52-0464-40f5-aff3-37009b78ad61', 'date_created': '2021-12-07T21:36:47.034Z', 'user_updated': 'eb3d7a52-0464-40f5-aff3-37009b78ad61', 'date_updated': '2022-03-10T00:01:34.263Z', 'arca_id': 131, 'nombre': 'Santiago (Cl)', 'pais': 22}
Creada relacion ubicacion pais
{'id': 683, 'status': 'draft', 'sort': None, 'user_created': 'eb3d7a52-0464-40f5-aff3-37009b78ad61', 'date_created': '2021-12-07T21:40:02.089Z', 'user_updated': 'eb3d7a52-0464-40f5-aff3-37009b78ad61', 'date_updated': '2022-03-10T00:02:39.634Z', 'arca_id': 683, 'nombre': 'Acapulco', 'pais': 17}
econtrada ubicacion: 627
{'id': 17, 'status': 'draft', 'sort': No